In [ ]:
import pandas as pd
import numpy as np

from scipy.sparse import kronsum

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure, text
import networkx as nx

## Generic functions

In [ ]:
def exp_kernel(train, sigma):
    """
    Computes the exponential kernel matrix for the given data and sigma.
    
    Parameters:
    train (np.array or pd.DataFrame): The input data matrix.
    sigma (float): The kernel bandwidth parameter.
    
    Returns:
    pd.DataFrame: The computed kernel matrix, rounded to 6 decimal places.
    """
    matrix_train = np.exp(-(train**2) / (2 * (sigma**2)))

    x = pd.DataFrame(matrix_train)
    x = np.round(x, 6)

    eigenvalues = np.linalg.eig(x)
    matrix = np.array(x)
    if not (np.sum(np.abs(eigenvalues[0]) > 0) == x.shape[0]) and (np.array_equal(matrix, matrix.T)):
        print("==============================")
        print("DOES NOT satisfy kernel condition")
        print("==============================")

    return x


def buildAndExportGraph(shiftOperator, params):
    """
    Builds a weighted undirected graph from the shift operator matrix and node labels.
    
    Parameters:
    shiftOperator (np.array or pd.DataFrame): The shift operator matrix representing the graph.
    params (list of str): List of node labels corresponding to the matrix indices.
    
    Returns:
    networkx.Graph: The constructed weighted undirected graph.
    """
    aux = pd.DataFrame(data=shiftOperator, columns=params)
    aux.index = params

    # Create an empty graph
    G = nx.Graph()
    
    for i in range(aux.shape[0]):
        for j in range(aux.shape[0]):
            # Add edges iteratively with weights
            G.add_edges_from([(params[i], params[j])], weight=aux.iloc[i, j])
            
    return G


### Common Parameters

In [ ]:
norm = "normPower2"
numberOfTimeSteps = 14
debug_plot_figures = True

folders = ["s1", "s2", "s3"]

keys = ['AMG', 'ATF', 'ATI', 'ATP', 'CAR', 'CF1', 'CF2', 'CF3', 'CF4', 'Falta',
        'GCC', 'GLI', 'LIN', 'LIP', 'MAC', 'MON', 'NTI', 'OTR', 'OXA', 'PAP',
        'PEN', 'POL', 'QUI', 'SUL', 'TTC',
        'MV hours', '# pat$_{atb}$', '# pat$_{MDR}$',
        'CAR$_{n}$', 'PAP$_{n}$', 'Falta$_{n}$',
        'QUI$_{n}$', 'ATF$_{n}$', 'OXA$_{n}$', 'PEN$_{n}$',
        'CF3$_{n}$', 'GLI$_{n}$', 'CF4$_{n}$', 'SUL$_{n}$',
        'NTI$_{n}$', 'LIN$_{n}$', 'AMG$_{n}$', 'MAC$_{n}$',
        'CF1$_{n}$', 'GCC$_{n}$', 'POL$_{n}$', 'ATI$_{n}$',
        'MON$_{n}$', 'LIP$_{n}$', 'TTC$_{n}$', 'OTR$_{n}$',
        'CF2$_{n}$', 'ATP$_{n}$', 
        '# pat$_{tot}$',
        'Post change',
        'Insulin', 'Art nutrition', 'Sedation', 'Relax', 'Hepatic$_{fail}$',
        'Renal$_{fail}$', 'Coagulation$_{fail}$', 'Hemodynamic$_{fail}$',
        'Respiratory$_{fail}$', 'Multiorganic$_{fail}$',  '# transfusions',
        'Vasoactive drug', 'Dosis nems', 'Tracheo$_{hours}$', 'Ulcer$_{hours}$',
        'Hemo$_{hours}$', 'C01 PIVC 1',
        'C01 PIVC 2', 'C02 CVC - RJ',
        'C02 CVC - RS', 'C02 CVC - LS', 'C02 CVC - RF',
        'C02 CVC - LJ', 'C02 CVC - LF', '# catheters']

binary = ['AMG', 'ATF', 'ATI', 'ATP', 'CAR', 'CF1',
            'CF2', 'CF3', 'CF4', 'Falta', 'GCC', 'GLI', 'LIN', 'LIP', 'MAC',
            'MON', 'NTI', 'OTR', 'OXA', 'PAP', 'PEN', 'POL', 'QUI', 'SUL', 'TTC',
            'Post change',
            'Insulin', 'Art nutrition', 'Sedation', 'Relax', 'Hepatic$_{fail}$',
            'Renal$_{fail}$', 'Coagulation$_{fail}$', 'Hemodynamic$_{fail}$',
            'Respiratory$_{fail}$', 'Multiorganic$_{fail}$',  'Vasoactive drug']


continues =  [variable for variable in keys if variable not in binary]

### Threshold value

In [ ]:
# Based on the threshold value, you can choose between: 0.6, 0.725, 0.85 and 0.975
threshold_val_init = 0.975
th_folder = "th_0975"
save_results = True

# 1.DTW-HGD

In [ ]:
buildGraph = "dtw-hgd"
numberOfFeatures = 80

In [ ]:
threshold_val_init = 0.975

T = numberOfTimeSteps
F = 80

dicc_thresholds_eachTS = {}
values_to_smoothness_bysplit = []
for c in range(len(folders)):
    print("====> Folder:" + str(folders[c]) + " <====")
    
    thresholds_by_timeStep = []
    values_to_smoothness = []
    
    #  Creates the initial network (G) based on all data to calculate the fixed items
    nt = 0
    data = pd.read_csv("../E2_Standard-GCN/dtw_matrices/"+folders[c]+"/tr_AMR_"+norm+".csv")
    data = data.fillna(0)
    K = exp_kernel(data, 2.5)
    K = K - np.eye(K.shape[0])
    G = nx.from_numpy_array(np.array(K), create_using=nx.DiGraph)

    fixed_pos = nx.spring_layout(G, seed=42, k=1.5, scale=2.5) 

    print("Threshold value:", threshold_val_init)
    print("Number of non-zero values before applying the threshold:", np.count_nonzero(K))
    s = K.copy()
    min_value = s.min().min()
    max_value = s.max().max()
    s = (s - min_value) / (max_value - min_value)
    s[np.abs(s) < threshold_val_init] = 0
    edges_aft = np.count_nonzero(s)
    values_to_smoothness.append(edges_aft)
    print("Number of non-zero values after applying the threshold:", edges_aft)
    print("%:", (edges_aft * 100) / (numberOfFeatures * numberOfFeatures))
    thresholds_by_timeStep.append((edges_aft * 100) / (numberOfFeatures * numberOfFeatures))

    adjacency_matrix_sparse = np.array(s)
    df = pd.DataFrame(adjacency_matrix_sparse)
    df.to_csv("../E2_Standard-GCN/dtw_matrices/"+folders[c]+"/tr_AMR_"+norm+"_sparse.csv", index=False)

    if debug_plot_figures:
        %matplotlib inline
        plt.figure()
        fig, ax = plt.subplots(1, 1, figsize=(10, 10))
        img = ax.imshow(s, extent=[0, numberOfFeatures, numberOfFeatures, 0])
        ax.set_xticks(range(0, numberOfFeatures+1, 5))  
        ax.set_yticks(range(0, numberOfFeatures+1, 5))  
        ax.set_xticklabels(range(0, numberOfFeatures+1, 5), rotation=90, fontsize=23) 
        ax.set_yticklabels(range(0, numberOfFeatures+1, 5), fontsize=23)
        cbar = fig.colorbar(img, ax=ax, fraction=0.046, pad=0.04)
        cbar.ax.tick_params(labelsize=23)
        plt.tight_layout(pad=0)
        #plt.savefig("./Figures/" +th_folder +"/"+ folders[c] + "/"+buildGraph + "_AdjtoSpatioTemporal_"+str(nt)+".pdf", format='pdf', bbox_inches='tight')
        plt.show()

        # Create a NetworkX graph from the adjacency matrix
        G = nx.from_numpy_array(np.array(s), create_using=nx.DiGraph)

        edges, weights = zip(*nx.get_edge_attributes(G, 'weight').items()) if len(G.edges()) > 0 else ([], [])
        options = {
            "edgelist": edges,
            "edge_color": weights,
            "width": np.array(weights) * 1 if weights else [],
            "alpha": 1,
        }

        node_colors = ['lightblue' if keys[node] in binary else 'lightgreen' for node in G.nodes()]  # Soft colors
        node_sizes = [700 if G.degree(node) > 0 else 500 for node in G.nodes()]  # Larger size for nodes with links

        fig2, ax2 = plt.subplots(figsize=(16,8))
        nx.draw_networkx_nodes(G, fixed_pos, node_color=node_colors, node_size=node_sizes, ax=ax2)  # Draw all nodes with soft colors and sizes
        if edges:
            nx.draw_networkx_edges(G, fixed_pos, **options, ax=ax2)  # Draw only the present edges

        labels = {node: node for node in G.nodes()}
        nx.draw_networkx_labels(G, fixed_pos, labels=labels, font_size=14, font_weight='bold', ax=ax2)
        plt.tight_layout()
        #plt.savefig("./Figures/" +th_folder +"/"+ folders[c] + "/"+buildGraph + "_Graph_"+str(nt)+".pdf", format='pdf', bbox_inches='tight')
        plt.show()

